# Building Up Process Scheduling Program
#### Queue1 = Round Robin List take values that has High Priority = 0 with quantum time = 3
#### Queue2 = SRJ take values that has medium Priority = 1 with quantum time = 2
#### Qeuee3 = SJF take values that has low Priority = 2 without quantum time as it's non-premptive 
# Needed Inputs
#### 1.Process Name (Pi)
#### 2.Process Arival Time 
#### 3.Prcoess Execution Time
#### 4.Queue Number of Priority


In [13]:
def SRT(processData):
    n = len(processData)

    arrival_time = [process[1] for process in processData]
    burst_time = [process[2] for process in processData]
    waiting_time = [0] * n
    turnaround_time = [0] * n

    quantum_time = 2

    remaining_time = [0] * n
    for i in range(n):
        remaining_time[i] = burst_time[i]

    t = 0  # Current time
    completed = 0  # Number of completed processes
    while completed != n:
        shortest = -1
        for i in range(n):
            if arrival_time[i] <= t and remaining_time[i] > 0:
                if shortest == -1 or remaining_time[i] < remaining_time[shortest]:
                    shortest = i

        if shortest == -1:
            t += 1
            continue

        if remaining_time[shortest] > quantum_time:
            t += quantum_time
            remaining_time[shortest] -= quantum_time
        else:
            t += remaining_time[shortest]
            waiting_time[shortest] = t - burst_time[shortest] - arrival_time[shortest]
            turnaround_time[shortest] = t - arrival_time[shortest]
            remaining_time[shortest] = 0
            completed += 1

    total_waiting_time = sum(waiting_time)
    total_turnaround_time = sum(turnaround_time)
    average_waiting_time = total_waiting_time / n
    average_turnaround_time = total_turnaround_time / n

    print("\nProcess\t\tArrival Time\tBurst Time\tWaiting Time\tTurnaround Time")
    for i in range(n):
        print(
            "{}\t\t{}\t\t{}\t\t{}\t\t{}".format(
                processData[i][0],
                arrival_time[i],
                burst_time[i],
                waiting_time[i],
                turnaround_time[i],
            )
        )

    print("\nAverage Waiting Time:", average_waiting_time)
    print("Average Turnaround Time:", average_turnaround_time)

In [14]:
def SJF(processes):
    n = len(processes)

    # Sort the processes based on burst time
    sorted_processes = sorted(processes, key=lambda x: x[2])

    # Calculate waiting time for each process
    waiting_time = [0] * n
    for i in range(1, n):
        waiting_time[i] = sorted_processes[i - 1][2] + waiting_time[i - 1]

    # Calculate turnaround time for each process
    turnaround_time = [0] * n
    for i in range(n):
        turnaround_time[i] = sorted_processes[i][2] + waiting_time[i]

    # Calculate average waiting time and average turnaround time
    total_waiting_time = sum(waiting_time)
    total_turnaround_time = sum(turnaround_time)
    average_waiting_time = total_waiting_time / n
    average_turnaround_time = total_turnaround_time / n

    # Print the results
    print("Process\tBurst Time\tWaiting Time\tTurnaround Time")
    for i in range(n):
        process_id = sorted_processes[i][0]
        burst_time = sorted_processes[i][2]
        print(
            f"{process_id}\t\t{burst_time}\t\t{waiting_time[i]}\t\t{turnaround_time[i]}"
        )

    print(f"\nAverage Waiting Time: {round(average_waiting_time)}")
    print(f"Average Turnaround Time: {round(average_turnaround_time)}")


In [15]:
def process_data(number_of_processes):
    process_SRT = []
    process_SJF = []
    process_RR = []

    for i in range(number_of_processes):
        process_id = int(input("Enter Process ID: "))
        arrival_time = int(input(f"Enter Arrival Time for Process {process_id}: "))
        burst_time = int(input(f"Enter Burst Time for Process {process_id}: "))
        priority_rank = int(input(f"Enter priority rank for Process {process_id}: "))

        if priority_rank == 0:
            process_RR.append([process_id, arrival_time, burst_time, priority_rank])
        elif priority_rank == 1:
            process_SJF.append([process_id, arrival_time, burst_time, priority_rank])
        elif priority_rank == 2:
            process_SRT.append([process_id, arrival_time, burst_time, priority_rank])

    return process_RR,process_SRT, process_SJF


# Get user input for the number of processes
num_processes = int(input("Enter the number of processes: "))

# Call the function to get process data
rr,srt,sjf = process_data(num_processes)

In [16]:
rr

[[2, 1, 8, 0], [5, 16, 10, 0]]

In [17]:
srt

[[2, 3, 6, 2], [3, 5, 4, 2]]

In [18]:
sjf

[[0, 0, 5, 1], [4, 8, 2, 1]]

In [19]:
SRT(srt)


Process		Arrival Time	Burst Time	Waiting Time	Turnaround Time
2		3		6		0		6
3		5		4		4		8

Average Waiting Time: 2.0
Average Turnaround Time: 7.0


In [22]:
SJF(sjf)

Process	Burst Time	Waiting Time	Turnaround Time
4		2		0		2
0		5		2		7

Average Waiting Time: 1
Average Turnaround Time: 4


In [23]:
from collections import deque


def round_robin(process_data, time_quantum):
    total_turnaround_time = 0
    total_waiting_time = 0
    start_time = []
    exit_time = []
    remaining_burst_time = []
    queue = deque()
    s_time = 0

    for process in process_data:
        remaining_burst_time.append(process[2])

    while True:
        all_finished = True

        for i in range(len(process_data)):
            if process_data[i][1] <= s_time and remaining_burst_time[i] > 0:
                all_finished = False
                if remaining_burst_time[i] > time_quantum:
                    start_time.append(s_time)
                    s_time += time_quantum
                    remaining_burst_time[i] -= time_quantum
                    exit_time.append(s_time)
                    queue.append(process_data[i])
                else:
                    start_time.append(s_time)
                    s_time += remaining_burst_time[i]
                    exit_time.append(s_time)
                    remaining_burst_time[i] = 0

        if all_finished:
            break

    for i in range(len(process_data)):
        turnaround_time = exit_time[i] - process_data[i][1]
        total_turnaround_time += turnaround_time
        process_data[i].append(turnaround_time)

        waiting_time = turnaround_time - process_data[i][2]
        total_waiting_time += waiting_time
        process_data[i].append(waiting_time)

    average_turnaround_time = total_turnaround_time / len(process_data)
    average_waiting_time = total_waiting_time / len(process_data)

    print(
        "Process ID\tArrival Time\tBurst Time\tCompletion Time\tTurnaround Time\tWaiting Time"
    )
    for i in range(len(process_data)):
        print(
            f"{process_data[i][0]}\t\t{process_data[i][1]}\t\t{process_data[i][2]}\t\t{exit_time[i]}\t\t{process_data[i][3]}\t\t{process_data[i][4]}"
        )

    print(f"\nAverage Turnaround Time: {average_turnaround_time}")
    print(f"Average Waiting Time: {average_waiting_time}")


x = [[1, 0, 3], [2, 2, 9]]
time_quantum = 2
round_robin(x, time_quantum)

Process ID	Arrival Time	Burst Time	Completion Time	Turnaround Time	Waiting Time
1		0		3		2		2		-1
2		2		9		4		2		-7

Average Turnaround Time: 2.0
Average Waiting Time: -4.0
